# PIC-SURE R API use-case: Phenome-Wide analysis on BioData Catalyst studies

This notebook is an illustration example about how to query data using the R **PIC-SURE API**. It takes as use-case a simple PheWAS analysis. This notebook is intentionally straightforward, and explanation provided are only aimed at guiding through the PheWAS analysis process. For a more step-by-step introduction to the R PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to get an user-specific security token. For more information on how to proceed, see the `get_your_token.ipynb` notebook**

## Environment set-up

### System requirements
- R 4.0 or later

### Packages installation

In [ ]:
source("R_lib/requirements.R")

#### Installing latest R PIC-SURE API libraries from github

In [ ]:
library('devtools', lib.loc = '~/studies/avillachlab-picsure-libraries-all/lib/')

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
#install.packages("https://cran.r-project.org/src/contrib/Archive/devtools/devtools_1.13.6.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/R6_2.5.1.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/hash_2.2.6.1.tar.gz", repos=NULL, type="source")
install.packages(c("urltools"),repos = "http://cran.us.r-project.org")
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", force=T)

In [ ]:
########testing cell
library(dplyr)
library(ggplot2)

#### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE network

In [ ]:
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")


In [ ]:
myconnection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)

In [ ]:
resource <- bdc::get.resource(myconnection,
                               resourceUUID = resource_id)

# PheWAS analysis
*Note: This example is not meant to be publication-ready, but rather serve as a guide or starting point to perform PheWAS.*

This PheWAS analysis focuses on the TopMed DCC Harmonized Variables. 
We leverage the harmonized variables to provide an example PheWAS focused on total cholesterol in two studies: ARIC and FHS.
The PIC-SURE API is helpful in wrangling our phenotypic data. 

In a nutshell, this PheWAS analysis follows the subsequent steps:
1. Retrieving the variable dictionary, using the PIC-SURE API dedicated methods
2. Using the PIC-SURE API to select variables and retrieve data
3. Data management
4. Statistical analysis for each study and sex
5. Visualization of results in Manhattan Plot

With this, we are tackling two different analysis considerations of a PheWAS: 
1) Using multiple variables in a PheWAS
2) Harmonization and meta-analysis issues when using data from multiple studies or datasets

### 1. Retrieving variable dictionary from PIC-SURE
The first step to conducting the PheWAS is to retrieve information about the variables that will be used in the analysis. For this example, we will be using variables from the TOPMed Data Coordinating Center (DCC) Harmonized data set. 

The Data Harmonization effort aims to produce "a high quality, lasting resource of publicly available and thoroughly documented harmonized phenotype variables". The TOPMed DCC collaborates with Working Group members and phenotype experts on this endeavour. So far, 44 harmonized variables are accessible through PIC-SURE API (in addition to the age at which each variable value has been collected for a given subject).

Which phenotypic characteristics are included in the harmonized variables?

- Key NHLBI phenotypes
    - Blood cell counts
    - VTE
    - Atherosclerosis-related phenotypes
    - Lipids
    - Blood pressure
    
    
- Common covariates
    - Height
    - Weight
    - BMI
    - Smoking status
    - Race/ethnicity

More information about the variable harmonization process is available at https://www.nhlbiwgs.org/sites/default/files/pheno_harmonization_guidelines.pdf

Here, we retrieve the harmonized variables information by searching for `DCC Harmonized data set` in PIC-SURE.

In [ ]:
harmonized_variables <- bdc::find.in.dictionary(resource, "DCC Harmonized data set") 

In [ ]:
harmonized_dic <- lapply(bdc::extract.entries(harmonized_variables), as.character)
plain_variablesDict <- dplyr::bind_rows(harmonized_dic) %>% dplyr::arrange(name)

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict <- get_multiIndex_variablesDict(plain_variablesDict)
dim(variablesDict)
head(variablesDict)

### 2. Using the PIC-SURE API to select variables and retrieve data
As you can see, we retrieved 80 variables from the DCC harmonized data set. The structure of `variablesDict` allows us to visualize the tree-like structure of the concept paths more easily.
Now that we've retrieved the variable information, we need to select our variable of interest. In this example, we are interested in exploring the relationship between the harmonized variables and blood cholesterol. Specifically, we will find the full concept path that contains "Blood mass concentration of total cholesterol".

In [ ]:
my_harmonized_dic <- bdc::extract.entries(harmonized_variables)

In [ ]:
# Retrieve the dependent variable - total cholesterol
cholesterol_path <- my_harmonized_dic %>% filter(grepl('Blood mass concentration of total cholesterol', name)) %>% pull(name)
cholesterol_path

In [ ]:
# Create full list of concept paths with cholesterol_path removed
selected_vars <- lapply(variablesDict[["name"]] %>% as.list(), as.character)
selected_vars <- selected_vars[selected_vars != cholesterol_path]

We are ready to create our query and retrieve the dataframe. This query will consist of two parts:
1. **Any record of `cholesterol_path`.** By performing an "any record of" filter on the `cholesterol_path`, we will filter out all participants that do not have total blood cholesterol measurements. This allows us to perform more meaningful statistical analysis on the data.
2. **Select all remaining harmonized variables.** We will then add all of the remaining harmonized variables to the query, which will allow us to retrieve this information.

In [ ]:
# Initialize a query
query <- bdc::new.query(resource = resource)

In [ ]:
bdc::query.anyof.add(query, cholesterol_path) # Use anyof for the cholesterol variable to filter out the NA values
bdc::query.select.add(query, selected_vars)
facts <- bdc::query.run(query, result.type = "dataframe")

In [ ]:
head(facts)

### 3. Data-management
Now that we have retrieved the data, we shall perform some data management steps to prepare for the statistical analysis. First, we will identify which variables are categorical and which are continuous using the "categorical" column of the `facts` dataframe. This is an example of how the PIC-SURE API greatly simplifies this step for the user, as categorizing variables can be tricky.

In [ ]:
categorical_varnames <- my_harmonized_dic %>% filter(categorical == TRUE) %>% pull(name)
continuous_varnames <-  my_harmonized_dic %>% filter(categorical == FALSE) %>% pull(name)

In [ ]:
# remove cholesterol_path from continuous_varnames
continuous_varnames <- continuous_varnames[-which(continuous_varnames == cholesterol_path)]
# remove subgroup concept path from categorical_varnames
categorical_varnames <- categorical_varnames[-which(categorical_varnames == "\\DCC Harmonized data set\\01 - Demographics\\A distinct subgroup within a study  generally indicating subjects who share similar characteristics due to study design. Subjects may belong to only one subcohort.\\")]

To perform this PheWAS, we will frame two participant cohorts in the context of the dependent variable of interest. In this example, we are interested in blood cholesterol. However, `Blood mass concentation of total cholesterol` is a continuous variable. We shall convert this variable into a binary variable with two groups, Normal/Low and High cholesterol levels, by applying a [threshold of 200mg/dL](https://www.mayoclinic.org/diseases-conditions/high-blood-cholesterol/diagnosis-treatment/drc-20350806). 

In [ ]:
chol_vector <- as.numeric(unlist(facts['\\DCC Harmonized data set\\06 - Lipids\\Blood mass concentration of total cholesterol\\']))
facts$categorical_cholesterol <- cut(chol_vector, breaks = c(48, 200, 647),
                  labels = c("Normal/Low", "High"))

We will also specify the variable name for the covariate we are interested in, in this case Sex.

In [ ]:
sex_path <- colnames(facts)[grep('sex', colnames(facts))]
sex_path

We will also select our cohorts of interest. In this example, we are interested in participants from the Framingham Heart Study (FHS) and the Atherosclerosis Risk In Communities (ARIC) cohort. We can utilize the `A distinct subgroup within a study  generally indicating subjects who share similar characteristics due to study design. Subjects may belong to only one subcohort.` concept path in the DCC Harmonized data set to select the participants of interest.

In [ ]:
subgroup_var <- colnames(facts)[2] # Get concept path for distinct subgroups
# Filter using FHS and ARIC
fhs_subset <- facts %>% filter(grepl('FHS', get(subgroup_var)))
aric_subset <- facts %>% filter(grepl('ARIC', get(subgroup_var)))

### 4. Statistical analysis
Two different association tests will be carried out according to variables data types:
- Logistic regression for continuous variables, using the `glm` R function
- Fisher exact test for categorical variables, using the `fisher.test` R function

We will create two functions, `test_continuous` and `test_categorical`, to perform these statistical tests. 
An additional function, `check_vars`, will be used to check if the data passes some assumptions of these tests.


In [ ]:
test_continuous <- function(dependent_vec, independent_vec){
    
    pval <- 999
    glm.fit <- glm(dependent_vec ~ independent_vec, family = 'binomial')
    try(pval <- coef(summary(glm.fit))['independent_vec','Pr(>|z|)'], silent = TRUE)
    if (pval == 999) {pval <- NA}
    
    return(pval)
}

test_categorical <- function(dependent_vec, independent_vec) {
    
    contingency_table <- table(dependent_vec, independent_vec)
    
    # set arbitrary cutoff (5): do not compute stats for categorical variables with too many levels
    if (max(dim(contingency_table)) > 5 | min(dim(contingency_table)) == 1) {
        return(NA)
    } else {
        res <- fisher.test(contingency_table, workspace = 2e8, simulate.p.value=TRUE)
        pval <- res$p.value
        return(pval)
    }   
}

check_vars <- function(dependent_var, other_var, df, case_value, control_value){
    pass <- FALSE
    cases <- df %>% filter(get(dependent_var) == case_value)
    controls <- df %>% filter(get(dependent_var) == control_value)
    
    
    try(if ((var(cases, na.rm = TRUE) != 0) & (var(controls, na.rm = TRUE) != 0)) {pass <- TRUE}, silent = TRUE)
    
    concept_vec <- df %>% pull(eval(other_var))
    if(length(table(concept_vec)) > 1) {pass <- TRUE}
    
    return(pass)
}

We wrap the previously created functions into one broad analysis function: ```run_phewas```. This would allow the user to run a variation of the analyses described here simply by modifying the calls to the function. The arguments are described below.
- facts: a dataframe representing the results from your PIC-SURE query. This dataframe can be filtered as needed.
- dependent_var: the column name corresponding to the name of your outcome varible. In this example. 'categorical_cholesterol'.
- continuous_varnames: a vector containing all column names of continuous variables within the facts dataframe which you would like to test.
- categorical_varnames: a vector containing all column names of categorical variables within the facts dataframe which you would like to test.
- case_value: the value corresponding to the 'cases' in the dependent_var vector
- control_value: the value corresponding to the 'controls' in the dependent_var vector

In [ ]:
run_phewas <- function(facts, dependent_var, continuous_varnames, categorical_varnames, case_value, control_value){

    results_df <- data.frame('concept_code' = '', 
                             'simplified_varname' = '', 
                             'vartype' = '', 
                             'pval' = 1.0,
                             'n_cases' = 0,
                             'n_controls' = 0,
                             'var_cases' = 0,
                             'var_controls' = 0
                            )[-1,]

    for (other_var in continuous_varnames){
        
        df <- facts %>% select(eval(dependent_var), eval(other_var)) # do any filtering, removal of NA or blank values here if necessary (currently not doing so)
        check <- check_vars(dependent_var, other_var, df, case_value, control_value)
        if(check){
        results_df <- results_df %>% add_row(concept_code = other_var, 
                                             simplified_varname = str_extract(other_var, '\\\\[^\\\\]*\\\\$') %>% str_replace_all('\\\\', ''), 
                                             vartype = 'continuous', 
                                             pval = test_continuous(df[,1], df[,2]),
                                             n_cases = df %>% filter(get(dependent_var) == case_value, !is.na(get(other_var))) %>% nrow(),
                                             n_controls = df %>% filter(get(dependent_var) == control_value, !is.na(get(other_var))) %>% nrow(),
                                             var_cases = df %>% filter(get(dependent_var) == case_value) %>% pull(eval(other_var)) %>% var(na.rm = TRUE), #Variance
                                             var_controls = df %>% filter(get(dependent_var) == control_value) %>% pull(eval(other_var)) %>% var(na.rm = TRUE)
                                            )
            }
    }
    for (other_var in categorical_varnames){
        
        df <- facts %>% select(eval(dependent_var), eval(other_var)) # do any filtering, removal of NA or blank values here if necessary (currently not doing so)
        check <- check_vars(dependent_var, other_var, df, case_value, control_value)
        if(check) {
        results_df <- results_df %>% add_row(concept_code = other_var, 
                                             simplified_varname = str_extract(other_var, '\\\\[^\\\\]*\\\\$') %>% str_replace_all('\\\\', ''), 
                                             vartype = 'categorical', 
                                             pval = test_categorical(df[,2], df[,1]),
                                             n_cases = df %>% filter(get(dependent_var) == case_value, !is.na(get(other_var))) %>% nrow(),
                                             n_controls = df %>% filter(get(dependent_var) == control_value, !is.na(get(other_var))) %>% nrow(),
                                             var_cases = NA,
                                             var_controls = NA
                                            )
        
            }
        }
    return(results_df)
}

We then use our previously defined wrapper function to run the phewas 4 times:
- Testing all harmonized variables against cholesterol in females in the FHS study
- Testing all harmonized variables against cholesterol in males in the FHS study
- Testing all harmonized variables against cholesterol in females in the ARIC study
- Testing all harmonized variables against cholesterol in males in the ARIC study

In [ ]:
fhs_female_df <- run_phewas(fhs_subset %>% filter(get(sex_path) == 'Female'),
                                     "categorical_cholesterol", 
                                     continuous_varnames, 
                                     categorical_varnames, 
                                     case_value="High", 
                                     control_value="Normal/Low") %>%
    mutate(sex = 'Female',
           study = 'FHS')

fhs_male_df <- run_phewas(fhs_subset %>% filter(get(sex_path) == 'Male'),
                                     "categorical_cholesterol", 
                                     continuous_varnames, 
                                     categorical_varnames, 
                                     case_value="High", 
                                     control_value="Normal/Low") %>%
    mutate(sex = 'Male',
           study = 'FHS')

aric_female_df <- run_phewas(aric_subset %>% filter(get(sex_path) == 'Female'), 
                                       "categorical_cholesterol", 
                                       continuous_varnames, 
                                       categorical_varnames, 
                                       case_value="High", 
                                       control_value="Normal/Low") %>%
    mutate(sex = 'Female',
           study = 'ARIC')

aric_male_df <- run_phewas(aric_subset %>% filter(get(sex_path) == 'Male'), 
                                       "categorical_cholesterol", 
                                       continuous_varnames, 
                                       categorical_varnames, 
                                       case_value="High", 
                                       control_value="Normal/Low") %>%
    mutate(sex = 'Male',
           study = 'ARIC')

In [ ]:
combined_df <- rbind(fhs_female_df, fhs_male_df, aric_female_df, aric_male_df)
head(combined_df)

Because we are running many statistical tests, we need to perform a p-value adjustment. Here, we use the holm-bonferroni method with an alpha of 0.01.

In [ ]:
combined_df$adj_pvalues <- p.adjust(combined_df$pval, method="holm")
combined_df$log_adj_pvalues <- -log10(combined_df$adj_pvalues)
adjusted_alpha <- -log10(0.01) 

In [ ]:
# idenfity top 5 adjusted p value results for each sex / study category
combined_df <- combined_df %>% 
    group_by(study, sex) %>% 
    mutate(top5pbystudy = log_adj_pvalues %in% sort(log_adj_pvalues, decreasing = TRUE)[1:5])

### 5. Visualization of results in a Manhattan plot
We plot a Manhattan plot, commonly used in PheWAS analyses, to visualize our results.
The x axis represents each of the phenotypes tested, and the y axis represents their associated -log10 p value. 

In [ ]:
options(repr.plot.width=25, repr.plot.height=20)
library(ggplot2)
my_theme <- theme_bw() +
theme(axis.title.y = element_text(face="italic", size=30),
      title = element_text(size=35),
      axis.title.x = element_text(size=30),
      axis.text.x = element_blank(),
      #legend.position = "none",
      panel.grid.major.x = element_blank()
      )

In [ ]:
# Manatthan plot using geom_jitter --> more suitable when number of point per category is enough
ggplot(combined_df, aes(x=simplified_varname, y=log_adj_pvalues)) +
geom_jitter(alpha=1, aes_string(colour="sex"),
           width=0.5,
           size=5) +
geom_hline(yintercept=adjusted_alpha, linetype="dashed", color = 'red') +
scale_y_continuous(expand = c(0, 20) ) +
geom_text(aes(label = ifelse(top5pbystudy, str_wrap(as.character(simplified_varname), 40),'')), hjust=.08, vjust=-0.08, size = 6) +
labs(title="Association between phenotypic variables and cholesterol level status",
    x="Phenotype",
    y="- log10(p-values)",
    colour="Variable type") +
facet_wrap(facets = vars(study), nrow = 2) +
my_theme


## PheWAS analysis

In a nutshell, this PheWAS analysis follows the subsequent steps:
- Retrieving the variables dictionary, using the PIC-SURE API dedicated methods
- From the info provided by the dictionary, retrieving the data in an exploitable format through PIC-SURE API calls
- Data management
- Running univariate tests again every phenotype variables
- Adjusting for multiple hypotheses testing issue
- Plotting results


This analysis is conducted using individuals enrolled in the COPDGene Study. Overall goal of this cohort is to detect underlying genetic factors to develop Chronic Obstructive Pulmonary Disease (COPD), and currently includes more than 10,000 individuals ([more information on COPDGene Study](http://www.copdgene.org)).

### 1. Retrieving variable dictionary from the Database

Retrieving variables dictionary for COPDGene related variables.

In [ ]:
copdgene_variables <- bdc::find.in.dictionary(resource, "Genetic Epidemiology of COPD (COPDGene)") 


In [ ]:
copdgene_dic <- lapply(bdc::extract.entries(copdgene_variables), as.character)
plain_variablesDict <- dplyr::bind_rows(copdgene_dic) %>% dplyr::arrange(name)

In [ ]:
my_copdgene_dic <- bdc::extract.entries(copdgene_variables)

Here we retrieved information about all the available variables of the COPDGene study, as well as information about the individual consents. The Consent variable will be eventually used to filter the patients pertaining to the COPDGene study. We are then combining both dictionaries into a single one.

In [ ]:
# Display the variables tree hierarchy from the variables name
variablesDict <- get_multiIndex_variablesDict(plain_variablesDict)
head(variablesDict)

### 2. Selecting variables and retrieving data from HPDS

Subseting to keep only the phenotype variables + the variable "affection status" that will be used as the dependent variable for this illustration use-case.

In [ ]:
# all copd variables
mask_copdgene <- variablesDict["level_0"] == 'Genetic Epidemiology of COPD (COPDGene) Funded by the National Heart, Lung, and Blood Institute ( phs000179 )'
# all phenotypic variables, determined by ? previosly called 'subject phenotype'
mask_pheno <- variablesDict["level_1"] == "Subject ID, died center, age at enrolment, race, ethnic, gender, body weight, body height, BMI, systolic and diastolic blood pressure, measurement of several parameters during 6 minutes work, CT slicer, CT scanner, heart rate, oxygen saturation and therapy, medical history of back pain, cancer, cardio vascular diseases, diabetes, digestive system diseases, eye diseases, general health, musculoskeletal diseases, painful joint type, respiratory tract disease, smoking, and walking limbs, medication history of treatment with beta-agonist, theophylline, inhaled corticosteroid, Oral corticosteroids, ipratropium bromide, and tiotroprium bromide, respiratory disease, St. George's Respiratory Questionnaire, SF-36 Health Survey, spirometry, and VIDA of participants with or without chronic obstructive pulmonary disease and involved in the 'Genetic Epidemiology of COPD (COPDGene) Funded by the National Heart, Lung, and Blood Institute' project."#'Subject Phenotype'
mask_status <- variablesDict["level_2"] == 'Affection status'
# Will probably need to add more to the mask_to_drop variable
mask_to_drop <- variablesDict[["simplified_name"]] %in% list("Dbgap_id", "De-identified site code", "A1AD: phenotype/genotype")
mask_vars <- mask_copdgene & (mask_pheno | mask_status) & !mask_to_drop
selected_vars <- variablesDict[mask_vars, ]$name %>% unname() %>% as.list()


### Subsetting only COPDGene records

We are using the COPDGene study accession 'phs000179'.

We can view the list of consents we have access to:

In [ ]:
#not sure if this is the best way to view all consents we have access to?
consent_variable <- bdc::find.in.dictionary(resource, "_parent_consents")  
consent_dic <- bdc::extract.entries(consent_variable)
available_consents <- unlist(consent_dic$categoryValues)
available_consents

In [ ]:
##### Note from Emily:
# May be able to use this? Got this from the 101 notebook - if we want to remain consistent
# bdc::query.show(bdc::new.query(resource = resource))

We will narrow our query to only include consent codes starting with 'phs000179'.


In [ ]:
copd_consents <- available_consents[grepl('phs000179', available_consents)]
copd_consents

next, we create a new query object: 

In [ ]:
my_query = bdc::new.query(resource = resource)

In [ ]:
# Delete existing consents
bdc::query.filter.delete(query = my_query,
                      keys = "\\_consents\\")
# add in our copd consents
bdc::query.filter.add(query = my_query,
                      keys = "\\_consents\\",
                      as.list(copd_consents))
# note: running this cell results in the following warning:
#    Warning message in grep(catval, ch):
#    “argument 'pattern' has length > 1 and only the first element will be used”
# but still functions as expected

In [ ]:
bdc::query.show(my_query)

In [ ]:
facts = bdc::query.run(query = my_query, result.type = "dataframe")

In [ ]:
head(facts)

In [ ]:
# add variables?
q = bdc::new.query(resource = resource)
bdc::query.filter.delete(query = q,
                      keys = "\\_consents\\")

# add in our copd consents
bdc::query.filter.add(query = q,
                      keys = "\\_consents\\",
                      as.list(copd_consents))

bdc::query.select.add(query = q,
                      keys = as.list(my_copdgene_dic %>% pull(name)))

facts = bdc::query.run(query = q, result.type = "dataframe")

In [ ]:
head(facts)

### 3. Data-management

#### Selecting variables regarding their types

One important step in a PheWAS is to separate variables by type (e.g., categorical, continuous). This distinction is straightforward using the variables dictionary.

In [ ]:
head(my_copdgene_dic)

In [ ]:
categorical_varnames <- my_copdgene_dic %>% filter(categorical == TRUE) %>% pull(name)
continuous_varnames <- my_copdgene_dic %>% filter(categorical == FALSE) %>% pull(name)

### Selecting the dependent variable to study
Most of PheWAS studies use a genetic variant as the dependent variable to separate the population between cases and controls. However the population doesn't have to be dichotomized using a genetic variant, and any phenotypic variable could be used to run a PheWAS analysis (see for example [*Neuraz et al.*, 2013](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1003405)).

Here we will use the **COPD status** as the case-control variable to dichotomize the population in our analysis, and keep only the population subset containing relevant values for the COPD status (i.e. keeping "Case" and "Control" individuals, thus discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
# find the full path for the dependent variable: Affection status
my_copdgene_dic %>% filter(grepl('Affection status', name))
affection_status_path <- my_copdgene_dic %>% filter(grepl('Affection status', name)) %>% pull(name)

In [ ]:
affection_status_path

In [ ]:
# remove affection_status_path from categorical_varnames
categorical_varnames <- categorical_varnames[-which(categorical_varnames == affection_status_path)]

Then we subset our population regarding the relevant values for the COPD diagnosis variable (i.e. keeping "Case" and "Control" individuals, and discarding "Other", "Control, Exclusionary Disease", and null values).

In [ ]:
subset <- facts %>% filter(get(affection_status_path) %in% c('Case', 'Control'))

In [ ]:
table(subset %>% pull(affection_status_path))

### 4. Univariate statistical tests

From this point, each variable present in the `facts_dummies` dataset will be tested against the selected dependent variable, (ie presence or absence of COPD).

Two different association tests will be carried out according to variables data types:
- Mann-Whitney U test for continuous variables
- Fisher exact test for categorical variables

In [ ]:
test_continuous <- function(dependent_vec, independent_vec){
    
    res <- wilcox.test(dependent_vec~independent_vec)
    pval <- res$p.value
    
    return(pval)
}

In [ ]:
test_categorical <- function(dependent_vec, independent_vec) {
    
    contingency_table <- table(dependent_vec, independent_vec)
    
    # set arbitrary cutoff (5): do not compute stats for categorical variables with too many levels
    if (max(dim(contingency_table)) > 5 | min(dim(contingency_table)) == 1) {
        return(NA)
    } else {
        res <- fisher.test(contingency_table, workspace = 2e8, simulate.p.value=TRUE)
        pval <- res$p.value
        return(pval)
    }
    
}

In [ ]:
results_df <- data.frame('full_varpath' = '', 
                         'simplified_varname' = '', 
                         'vartype' = '', 
                         'pval' = 1.0,
                         'n_cases' = 0,
                         'n_controls' = 0,
                         'var_cases' = 0,
                         'var_controls' = 0,
                         'odds_ratio' = 0 #https://www.rdocumentation.org/packages/questionr/versions/0.7.5/topics/odds.ratio 
                        )[-1,]

for (var in continuous_varnames){
    
    df <- subset %>% select(eval(affection_status_path), eval(var)) # do any filtering, removal of NA or blank values here if necessary (currently not doing so)

    results_df <- results_df %>% add_row(full_varpath = var, 
                                         simplified_varname = str_extract(var, '\\\\[^\\\\]*\\\\$') %>% str_replace_all('\\\\', ''), 
                                         vartype = 'continuous', 
                                         pval = test_continuous(df[,2], df[,1]),
                                         n_cases = df %>% filter(get(affection_status_path) == 'Case', !is.na(get(var))) %>% nrow(),
                                         n_controls = df %>% filter(get(affection_status_path) == 'Control', !is.na(get(var))) %>% nrow(),
                                         var_cases = df %>% filter(get(affection_status_path) == 'Case') %>% pull(eval(var)) %>% var(na.rm = TRUE),
                                         var_controls = df %>% filter(get(affection_status_path) == 'Control') %>% pull(eval(var)) %>% var(na.rm = TRUE)
                                        )
    
}

for (var in categorical_varnames){
    
    df <- subset %>% select(eval(affection_status_path), eval(var)) # do any filtering, removal of NA or blank values here if necessary (currently not doing so)
    
    results_df <- results_df %>% add_row(full_varpath = var, 
                                         simplified_varname = str_extract(var, '\\\\[^\\\\]*\\\\$') %>% str_replace_all('\\\\', ''), 
                                         vartype = 'categorical', 
                                         pval = test_categorical(df[,2], df[,1]),
                                         n_cases = df %>% filter(get(affection_status_path) == 'Case', !is.na(get(var))) %>% nrow(),
                                         n_controls = df %>% filter(get(affection_status_path) == 'Control', !is.na(get(var))) %>% nrow(),
                                         var_cases = NA,
                                         var_controls = NA
                                        )
    
}

In [ ]:
results_df

## 5. Multiple hypotheses testing correction: Bonferroni Method

In order to handle the multiple testing problem (increase in the probability of getting statistically significant associations), we will use the Bonferroni correction method. Although many other multiple-comparison corrections exist, Bonferroni is the most straightforward to use, because it doesn't require assumptions about variables correlation. Other PheWAS analysis also use False Discovery Rate controlling procedures ([see](https://en.wikipedia.org/wiki/False_discovery_rate)).

In a nutshell, Bonferonni allows to calculate a corrected "statistical significant threshold" according to the number of test performed. Every p-value below this threshold will be deemed statistically significant.

In [ ]:
results_df$adj_pvalues <- p.adjust(results_df$pval, method="holm")

In [ ]:
results_df$log_adj_pvalues <- -log10(results_df$adj_pvalues)

In [ ]:
adjusted_alpha <- -log10(0.01)

## 6. Result visualisations: Manhattan plot

Manhattan plot is the classical way to plot the results of a PheWAS analysis. It plots every tested phenotypical variables on the X-axis, against their *-log(pvalue)* on the Y-axis. The horizontal line represents the adjusted significance level threshold.

#### Plotting the data

In [ ]:
options(repr.plot.width=25, repr.plot.height=20)

In [ ]:
library(ggplot2)

In [ ]:
my_theme <- theme_bw() +
theme(axis.title.y = element_text(face="italic", size=15),
      title = element_text(size=20),
      axis.title.x = element_text(size=15),
      axis.text.x = element_blank(),
      #legend.position = "none",
      panel.grid.major.x = element_blank()
      )


In [ ]:
# Manatthan plot using geom_jitter --> more suitable when number of point per category is enough
ggplot(results_df, aes(x=simplified_varname, y=log_adj_pvalues)) +
geom_jitter(alpha=1, aes_string(colour="vartype"),
           width=0.5,
           size=3) +
geom_hline(yintercept=adjusted_alpha, linetype="dashed", color = 'red') +
scale_y_continuous(expand = c(0, 20) ) +
geom_text(aes(label = ifelse(log_adj_pvalues > adjusted_alpha, str_wrap(as.character(simplified_varname), 20),'')), hjust=.08, vjust=-0.08, size = 5) +
labs(title="Association between phenotypic variables and COPD status",
    x="Phenotype",
    y="- log10(p-values)",
    colour="Variable type") +
my_theme



Overall, it appears that most of the tested phenotypes covariates are above the adjusted threshold of significant association. However, it is not that surprising given the nature of our dependent variable: a lot of the collected phenotypic variables are correlated to the COPD status.